##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Lite Model Maker로 음성 인식 모델 재학습


<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/speech_recognition"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/speech_recognition.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/speech_recognition.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/models/modify/model_maker/speech_recognition.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

이 colab 노트북에서는 [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker)를 사용하여 1초 사운드 샘플로 구어 또는 짧은 문구를 분류할 수 있는 음성 인식 모델을 훈련하는 방법을 배웁니다. Model Maker 라이브러리는 전이 학습을 사용하여 기존 TensorFlow 모델을 새 데이터세트로 재학습하여 샘플 데이터의 양과 학습에 필요한 시간을 줄입니다.

기본적으로, 이 노트북은 [음성 명령 데이터세트](https://github.com/tensorflow/tfjs-models/tree/master/speech-commands#speech-command-recognizer)의 단어 하위 집합(예: "up", "down", "left", "right")을 사용하여 모델([TFJS Speech Command Recognizer](https://www.tensorflow.org/datasets/catalog/speech_commands)의 BrowserFft)을 재학습합니다. 그런 다음 모바일 장치 또는 임베디드 시스템(예: Raspberry Pi)에서 실행할 수 있는 TFLite 모델을 내보냅니다. 또한 훈련된 모델을 TensorFlow SavedModel로 내보냅니다.

이 노트북은 또한 ZIP 파일로 Colab에 업로드된 WAV 파일의 사용자 지정 데이터세트를 허용하도록 설계되었습니다. 각 클래스에 대한 샘플이 많을수록 정확도는 더 높아지지만 전이 학습 프로세스는 사전 훈련된 모델의 요소 임베딩을 사용하기 때문에 각 클래스에 수십 개의 샘플만 있으면 여전히 상당히 정확한 모델을 얻을 수 있습니다.

**참고:** 훈련할 모델은 1초 샘플을 이용한 음성 인식에 최적화되어 있습니다. 보다 일반적인 오디오 분류(예: 다양한 유형의 음악 감지)를 수행하려는 경우, 대신 [이 Colab에 따라 오디오 분류기를 재훈련](https://colab.sandbox.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/audio_classification.ipynb)하는 것이 좋습니다.

기본 음성 데이터세트로 노트북을 실행하려면 Colab 도구 모음에서 **Runtime(런타임) &gt; Run all(모두 실행)**을 클릭하여 지금 전체를 실행할 수 있습니다. 그러나 고유한 데이터세트를 사용하려면 [데이터세트 준비](#scrollTo=cBsSAeYLkc1Z)로 계속 진행하고 지침을 따르세요.


### 필요한 패키지 가져오기


TensorFlow, TFLite Model Maker, 그리고 오디오 조작, 재생 및 시각화를 위한 일부 모듈이 필요합니다.

In [ ]:
!sudo apt -y install libportaudio2
!pip install tflite-model-maker

In [ ]:
import os
import glob
import random
import shutil

import librosa
import soundfile as sf
from IPython.display import Audio
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tflite_model_maker as mm
from tflite_model_maker import audio_classifier
from tflite_model_maker.config import ExportFormat

print(f"TensorFlow Version: {tf.__version__}")
print(f"Model Maker Version: {mm.__version__}")

## 데이터세트 준비하기

기본 음성 데이터세트로 훈련하려면 아래의 모든 코드를 있는 그대로 실행하세요.

그러나 자신의 음성 데이터세트로 훈련하려면 다음 단계를 따르세요.

**참고:** 재학습할 모델은 입력 데이터가 44.1kHz의 오디오 약 1초일 것으로 예상합니다. Model Maker는 훈련 데이터세트에 대해 자동 리샘플링을 수행하므로 샘플 속도가 44.1kHz가 아닌 경우 데이터세트를 리샘플링할 필요가 없습니다. 그러나 1초보다 긴 오디오 샘플은 1초 단위의 여러 청크로 분할되며 1초보다 짧으면 최종 청크가 삭제됩니다.

1. 데이터세트의 각 샘플이 **약 1초 길이의 WAV 파일 형식**인지 확인하세요. 그런 다음 각 분류에 대해 별도의 하위 폴더로 구성된 모든 WAV 파일이 포함된 ZIP 파일을 만듭니다. 예를 들어 음성 명령 "yes"에 대한 각 샘플은 "yes"라는 하위 폴더에 있어야 합니다. 클래스가 하나만 있더라도 클래스 이름을 디렉터리 이름으로 사용하여 샘플을 하위 디렉터리에 저장해야 합니다. (이 스크립트는 데이터세트가 학습/검증/테스트 세트로 **분할되지 않았고** 이 분할을 자동으로 수행한다고 가정합니다.)
2. 왼쪽 패널에서 **Files(파일)** 탭을 클릭하고 거기에 ZIP 파일을 끌어다 놓아 업로드합니다.
3. 다음 드롭다운 옵션을 사용하여 **`use_custom_dataset`**을 True로 설정합니다.
4. 그런 다음 [사용자 지정 오디오 데이터세트 준비](#scrollTo=EobYerLQkiF1)로 건너뛰고 ZIP 파일 이름과 데이터세트 디렉터리 이름을 지정합니다.


In [ ]:
use_custom_dataset = False #@param ["False", "True"] {type:"raw"}

### 배경 소음 데이터세트 생성하기

기본 음성 데이터세트를 사용하든 사용자 지정 데이터세트를 사용하든 모델이 음성을 다른 소음(무음 포함)과 구별할 수 있도록 배경 소음이 잘 설정되어 있어야 합니다.

다음 배경 샘플은 길이가 1분 이상인 WAV 파일로 제공되므로 테스트 데이터세트용으로 일부를 예약할 수 있도록 더 작은 1초 샘플로 분할해야 합니다. 또한 몇 가지 다른 샘플 소스를 결합하여 포괄적인 배경 소음 및 무음 세트를 구축합니다.

In [ ]:
tf.keras.utils.get_file('speech_commands_v0.01.tar.gz',
                        'http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz',
                        cache_dir='./',
                        cache_subdir='dataset-speech',
                        extract=True)
tf.keras.utils.get_file('background_audio.zip',
                        'https://storage.googleapis.com/download.tensorflow.org/models/tflite/sound_classification/background_audio.zip',
                        cache_dir='./',
                        cache_subdir='dataset-background',
                        extract=True)


**참고:** 사용 가능한 최신 버전이 있지만 더 작은 다운로드이기 때문에 v0.01의 음성 명령 데이터세트를 사용합니다. v0.01에는 30개의 명령이 포함되어 있는 반면 v0.02에는 5개("backward", "forward", "follow", "learn" 및 "visual")가 추가되었습니다.

In [ ]:
# Create a list of all the background wav files
files = glob.glob(os.path.join('./dataset-speech/_background_noise_', '*.wav'))
files = files + glob.glob(os.path.join('./dataset-background', '*.wav'))

background_dir = './background'
os.makedirs(background_dir, exist_ok=True)

# Loop through all files and split each into several one-second wav files
for file in files:
  filename = os.path.basename(os.path.normpath(file))
  print('Splitting', filename)
  name = os.path.splitext(filename)[0]
  rate = librosa.get_samplerate(file)
  length = round(librosa.get_duration(filename=file))
  for i in range(length - 1):
    start = i * rate
    stop = (i * rate) + rate
    data, _ = sf.read(file, start=start, stop=stop)
    sf.write(os.path.join(background_dir, name + str(i) + '.wav'), data, rate)

### 음성 명령 데이터세트 준비하기

이미 음성 명령 데이터세트를 다운로드했으므로 이제 모델에 대한 클래스 수를 줄이기만 하면 됩니다.

이 데이터세트에는 30개 이상의 음성 명령 분류가 포함되어 있으며 대부분은 2,000개 이상의 샘플을 가지고 있습니다. 그러나 우리는 전이 학습을 사용하기 때문에 그렇게 많은 샘플이 필요하지 않습니다. 따라서 다음 코드는 몇 가지 작업을 수행합니다.

- 사용할 분류를 지정하고 나머지는 삭제합니다.
- 훈련을 위해 각 클래스의 샘플 150개만 유지합니다(전이 학습이 더 작은 데이터세트에서 잘 작동한다는 것을 증명하고 단순히 훈련 시간을 줄이기 위해).
- 나중에 쉽게 추론을 실행할 수 있도록 테스트 데이터세트에 대해 별도의 디렉터리를 만듭니다.

In [ ]:
if not use_custom_dataset:
  commands = [ "up", "down", "left", "right", "go", "stop", "on", "off", "background"]
  dataset_dir = './dataset-speech'
  test_dir = './dataset-test'

  # Move the processed background samples
  shutil.move(background_dir, os.path.join(dataset_dir, 'background'))   

  # Delete all directories that are not in our commands list
  dirs = glob.glob(os.path.join(dataset_dir, '*/'))
  for dir in dirs:
    name = os.path.basename(os.path.normpath(dir))
    if name not in commands:
      shutil.rmtree(dir)

  # Count is per class
  sample_count = 150
  test_data_ratio = 0.2
  test_count = round(sample_count * test_data_ratio)

  # Loop through child directories (each class of wav files)
  dirs = glob.glob(os.path.join(dataset_dir, '*/'))
  for dir in dirs:
    files = glob.glob(os.path.join(dir, '*.wav'))
    random.seed(42)
    random.shuffle(files)
    # Move test samples:
    for file in files[sample_count:sample_count + test_count]:
      class_dir = os.path.basename(os.path.normpath(dir))
      os.makedirs(os.path.join(test_dir, class_dir), exist_ok=True)
      os.rename(file, os.path.join(test_dir, class_dir, os.path.basename(file)))
    # Delete remaining samples
    for file in files[sample_count + test_count:]:
      os.remove(file)

### 사용자 지정 데이터세트 준비하기

자체 음성 데이터세트로 모델을 훈련하려면 [위에서 설명한 대로](#scrollTo=cBsSAeYLkc1Z) 샘플을 ZIP으로 압축한 WAV 파일로 업로드하고 다음 변수를 수정하여 데이터세트를 지정해야 합니다.

In [ ]:
if use_custom_dataset:
  # Specify the ZIP file you uploaded:
  !unzip YOUR-FILENAME.zip
  # Specify the unzipped path to your custom dataset
  # (this path contains all the subfolders with classification names):
  dataset_dir = './YOUR-DIRNAME'

위의 파일 이름과 경로 이름을 변경하면 사용자 지정 데이터세트로 모델을 학습할 준비가 된 것입니다. Colab 도구 모음에서 **Runtime(런타임) &gt; Run all(모두 실행)**을 선택하여 전체 노트북을 실행합니다.

다음 코드는 새로운 배경 잡음 샘플을 데이터세트에 통합한 다음 모든 샘플의 일부를 분리하여 테스트 세트를 만듭니다.

In [ ]:
def move_background_dataset(dataset_dir):
  dest_dir = os.path.join(dataset_dir, 'background')
  if os.path.exists(dest_dir):
    files = glob.glob(os.path.join(background_dir, '*.wav'))
    for file in files:
      shutil.move(file, dest_dir)
  else:
    shutil.move(background_dir, dest_dir)

In [ ]:
if use_custom_dataset:
  # Move background samples into custom dataset
  move_background_dataset(dataset_dir)

  # Now we separate some of the files that we'll use for testing:
  test_dir = './dataset-test'
  test_data_ratio = 0.2
  dirs = glob.glob(os.path.join(dataset_dir, '*/'))
  for dir in dirs:
    files = glob.glob(os.path.join(dir, '*.wav'))
    test_count = round(len(files) * test_data_ratio)
    random.seed(42)
    random.shuffle(files)
    # Move test samples:
    for file in files[:test_count]:
      class_dir = os.path.basename(os.path.normpath(dir))
      os.makedirs(os.path.join(test_dir, class_dir), exist_ok=True)
      os.rename(file, os.path.join(test_dir, class_dir, os.path.basename(file)))
    print('Moved', test_count, 'images from', class_dir)

### 샘플 재생

데이터세트가 올바른지 확인하기 위해 테스트 세트에서 임의의 샘플을 재생해 보겠습니다.

In [ ]:
def get_random_audio_file(samples_dir):
  files = os.path.abspath(os.path.join(samples_dir, '*/*.wav'))
  files_list = glob.glob(files)
  random_audio_path = random.choice(files_list)
  return random_audio_path

def show_sample(audio_path):
  audio_data, sample_rate = sf.read(audio_path)
  class_name = os.path.basename(os.path.dirname(audio_path))
  print(f'Class: {class_name}')
  print(f'File: {audio_path}')
  print(f'Sample rate: {sample_rate}')
  print(f'Sample length: {len(audio_data)}')

  plt.title(class_name)
  plt.plot(audio_data)
  display(Audio(audio_data, rate=sample_rate))

In [ ]:
random_audio = get_random_audio_file(test_dir)
show_sample(random_audio)

## 모델 정의하기

Model Maker를 사용하여 모델을 재학습할 때 먼저 모델 사양을 정의해야 합니다. 사양은 새 모델이 새 클래스 학습을 시작하기 위해 기능 임베딩을 추출할 기본 모델을 정의합니다. 이 음성 인식기의 사양은 [TFJS의 사전 훈련된 BrowserFft 모델](https://github.com/tensorflow/tfjs-models/tree/master/speech-commands#speech-command-recognizer)을 기반으로 합니다.

이 모델은 44.1kHz의 오디오 샘플이 1초 미만의 길이로 입력될 것을 예상합니다. 정확한 샘플 길이는 44034 프레임이어야 합니다.

훈련 데이터세트로 리샘플링을 수행할 필요가 없습니다. Model Maker가 알아서 처리해 줍니다. 그러나 나중에 추론을 실행할 때 입력이 예상 형식과 일치하는지 확인해야 합니다.

여기서 해야 할 일은 [`BrowserFftSpec`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/BrowserFftSpec)을 인스턴스화하는 것입니다.


In [ ]:
spec = audio_classifier.BrowserFftSpec()

## 데이터세트 로드하기 

이제 모델 사양에 따라 데이터세트를 로드해야 합니다. Model Maker에는 폴더에서 데이터세트를 로드하고 모델 사양에 대한 예상 형식인지 확인하는 [`DataLoader`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/DataLoader) API가 포함되어 있습니다.

우리는 이미 일부 테스트 파일을 별도의 디렉터리로 이동하여 이를 예약했습니다. 그러면 나중에 추론을 더 쉽게 실행할 수 있습니다. 이제 훈련 세트, 검증 세트, 테스트 세트 등 각 분할에 대해 `DataLoader`를 생성합니다.

#### 음성 명령 데이터세트 로드하기


In [ ]:
if not use_custom_dataset:
  train_data_ratio = 0.8
  train_data = audio_classifier.DataLoader.from_folder(
      spec, dataset_dir, cache=True)
  train_data, validation_data = train_data.split(train_data_ratio)
  test_data = audio_classifier.DataLoader.from_folder(
      spec, test_dir, cache=True)

#### 사용자 지정 데이터세트 로드하기

**참고:** `cache=True` 설정은 훈련을 더 빠르게 하는 데 중요하지만(특히 데이터세트를 다시 샘플링해야 하는 경우) 데이터를 저장하는 데 더 많은 RAM이 필요합니다. 매우 큰 사용자 지정 데이터세트를 사용하는 경우, 캐싱이 RAM 용량을 초과할 수 있습니다.

In [ ]:
if use_custom_dataset:
  train_data_ratio = 0.8
  train_data = audio_classifier.DataLoader.from_folder(
      spec, dataset_dir, cache=True)
  train_data, validation_data = train_data.split(train_data_ratio)
  test_data = audio_classifier.DataLoader.from_folder(
      spec, test_dir, cache=True)


## 모델 학습


이제 Model Maker [`create()`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/create) 함수를 사용하여 모델 사양 및 훈련 데이터세트를 기반으로 모델을 만들고 훈련을 시작합니다.

사용자 지정 데이터세트를 사용하는 경우, 훈련 세트의 샘플 수에 맞게 배치 크기를 변경해야 할 수 있습니다.

**참고:** 첫 번째 epoch는 캐시를 생성해야 하므로 시간이 더 오래 걸립니다. 

In [ ]:
# If your dataset has fewer than 100 samples per class,
# you might want to try a smaller batch size
batch_size = 25
epochs = 25
model = audio_classifier.create(train_data, spec, validation_data, batch_size, epochs)

## 모델 성능 검토

위의 훈련 결과에서 정확도/손실이 좋아 보이더라도 모델이 아직 보지 못한 테스트 데이터를 사용하여 모델을 실행하는 것도 중요합니다. 여기서 `evaluate()` 메서드가 수행하는 작업이 바로 그것입니다.

In [ ]:
model.evaluate(test_data)

### 혼동 행렬 보기

이와 같은 분류 모델을 훈련할 때 [혼동 행렬](https://en.wikipedia.org/wiki/Confusion_matrix)을 검사하는 것도 유용합니다. 혼동 행렬은 분류기가 테스트 데이터의 각 분류에 대해 얼마나 잘 작동하는지에 대한 상세한 시각적 표현을 제공합니다.

In [ ]:
def show_confusion_matrix(confusion, test_labels):
  """Compute confusion matrix and normalize."""
  confusion_normalized = confusion.astype("float") / confusion.sum(axis=1)
  sns.set(rc = {'figure.figsize':(6,6)})
  sns.heatmap(
      confusion_normalized, xticklabels=test_labels, yticklabels=test_labels,
      cmap='Blues', annot=True, fmt='.2f', square=True, cbar=False)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

confusion_matrix = model.confusion_matrix(test_data)
show_confusion_matrix(confusion_matrix.numpy(), test_data.index_to_label)

## 모델 내보내기

마지막 단계는 모델을 모바일/임베디드 장치에서 실행하기 위해 TensorFlow Lite 형식으로 내보내고 다른 곳에서 실행하기 위해 [SavedModel](https://www.tensorflow.org/guide/saved_model) 형식으로 내보내는 것입니다.

Model Maker에서 `.tflite` 파일을 내보낼 때, 여기에는 나중에 추론하는 동안 도움이 될 수 있는 다양한 세부 정보를 설명하는 [모델 메타데이터](https://www.tensorflow.org/lite/inference_with_metadata/overview)가 포함됩니다. 여기에는 또한 분류 레이블 파일의 복사본도 포함되므로 별도의 `labels.txt` 파일이 필요하지 않습니다. (다음 섹션에서는 이 메타데이터를 사용하여 추론을 실행하는 방법을 보여줍니다.)

In [ ]:
TFLITE_FILENAME = 'browserfft-speech.tflite'
SAVE_PATH = './models'

In [ ]:
print(f'Exporing the model to {SAVE_PATH}')
model.export(SAVE_PATH, tflite_filename=TFLITE_FILENAME)
model.export(SAVE_PATH, export_format=[mm.ExportFormat.SAVED_MODEL, mm.ExportFormat.LABEL])

## TF Lite 모델로 추론 실행하기

이제 지원되는 [추론 라이브러리](https://www.tensorflow.org/lite/guide/inference) 또는 새로운 [TFLite AudioClassifier Task API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/audio_classifier)를 사용하여 TFLite 모델을 배포하고 실행할 수 있습니다. 다음 코드는 Python에서 `.tflite` 모델로 추론을 실행하는 방법을 보여줍니다.

In [ ]:
# This library provides the TFLite metadata API
! pip install -q tflite_support

In [ ]:
from tflite_support import metadata
import json

def get_labels(model):
  """Returns a list of labels, extracted from the model metadata."""
  displayer = metadata.MetadataDisplayer.with_model_file(model)
  labels_file = displayer.get_packed_associated_file_list()[0]
  labels = displayer.get_associated_file_buffer(labels_file).decode()
  return [line for line in labels.split('\n')]

def get_input_sample_rate(model):
  """Returns the model's expected sample rate, from the model metadata."""
  displayer = metadata.MetadataDisplayer.with_model_file(model)
  metadata_json = json.loads(displayer.get_metadata_json())
  input_tensor_metadata = metadata_json['subgraph_metadata'][0][
          'input_tensor_metadata'][0]
  input_content_props = input_tensor_metadata['content']['content_properties']
  return input_content_props['sample_rate']

모델이 실제 샘플로 얼마나 잘 작동하는지 관찰하려면 다음 코드 블록을 반복해서 실행하세요. 매번 새로운 테스트 샘플을 가져와서 추론을 실행하고 아래 오디오 샘플을 들을 수 있습니다.

In [ ]:
# Get a WAV file for inference and list of labels from the model
tflite_file = os.path.join(SAVE_PATH, TFLITE_FILENAME)
labels = get_labels(tflite_file)
random_audio = get_random_audio_file(test_dir)

# Ensure the audio sample fits the model input
interpreter = tf.lite.Interpreter(tflite_file)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_size = input_details[0]['shape'][1]
sample_rate = get_input_sample_rate(tflite_file)
audio_data, _ = librosa.load(random_audio, sr=sample_rate)
if len(audio_data) < input_size:
  audio_data.resize(input_size)
audio_data = np.expand_dims(audio_data[:input_size], axis=0)

# Run inference
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], audio_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

# Display prediction and ground truth
top_index = np.argmax(output_data[0])
label = labels[top_index]
score = output_data[0][top_index]
print('---prediction---')
print(f'Class: {label}\nScore: {score}')
print('----truth----')
show_sample(random_audio)

## TF Lite 모델 다운로드하기

이제 TF Lite 모델을 모바일 또는 임베디드 장치에 배포할 수 있습니다. 이전 추론 예제와 같이 대신 `.tflite` 파일 메타데이터에서 레이블을 검색할 수 있으므로 레이블 파일을 다운로드할 필요가 없습니다.

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download(tflite_file)

[Android](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/android/) 및 [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/ios)에서 TFLite 오디오 모델로 추론을 수행하는 엔드 투 엔드 예제 앱을 확인하세요.